In [84]:
import requests

In [85]:
from decouple import config

In [86]:
client_id = config('SPOTIPY_CLIENT_ID')
client_secret = config('SPOTIPY_CLIENT_SECRET')

In [87]:
AUTH_URL = 'https://accounts.spotify.com/api/token'

In [88]:
#POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret
})

In [89]:
auth_response_data = auth_response.json()

In [90]:
access_token = auth_response_data['access_token']

In [91]:
headers = {
    'Authorization': f'Bearer {access_token}'
}

In [92]:
BASE_URL = 'https://api.spotify.com/v1/'

In [93]:
track_id = '4qCfbDTCrFnfG5fYngu2wE'

In [94]:
r = requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers)

In [95]:
r = r.json()

In [96]:
r

{'danceability': 0.69,
 'energy': 0.721,
 'key': 4,
 'loudness': -8.437,
 'mode': 1,
 'speechiness': 0.0323,
 'acousticness': 0.32,
 'instrumentalness': 0.649,
 'liveness': 0.125,
 'valence': 0.357,
 'tempo': 104.027,
 'type': 'audio_features',
 'id': '4qCfbDTCrFnfG5fYngu2wE',
 'uri': 'spotify:track:4qCfbDTCrFnfG5fYngu2wE',
 'track_href': 'https://api.spotify.com/v1/tracks/4qCfbDTCrFnfG5fYngu2wE',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4qCfbDTCrFnfG5fYngu2wE',
 'duration_ms': 268120,
 'time_signature': 4}

In [97]:
radiohead_uri = '4Z8W4fKeB5YxbusRsdQVPb'

In [98]:
r = requests.get(BASE_URL + 'artists/' + radiohead_uri + '/albums', headers=headers, params={'include_groups': 'album', 'limit': 50})
d = r.json()

In [99]:
for album in d['items']:
    print(album['name'], '---', album['release_date'])

OK Computer OKNOTOK 1997 2017 --- 2017-06-23
A Moon Shaped Pool --- 2016-05-08
TKOL RMX 1234567 --- 2011-10-10
The King Of Limbs --- 2011-02-18
In Rainbows --- 2007-12-28
In Rainbows (Disk 2) --- 2007-12-03
Hail To the Thief --- 2003
Amnesiac --- 2001-03-12
I Might Be Wrong --- 2001
Kid A --- 2000-10-01
OK Computer --- 1997-05-28
The Bends --- 1995-03-28
Pablo Honey --- 1993-02-22


In [100]:
data = [] # will hold track info
albums = [] # to keep track of duplicates

#loop over albums and get all tracks
for album in d['items']:
    album_name = album['name']

    trim_name = album_name.split('(')[0].strip()
    if trim_name.upper() in albums or int(album['release_date'][:4]) > 1983:
        continue
    albums.append(trim_name.upper()) # use upper() to standardize

    print(album_name)

    #pull tracks from this album

    r = requests.get(BASE_URL + 'albums/' + album['id'] + '/tracks', headers=headers)
    tracks = r.json()['items']

    for track in tracks:
        #get features
        f = requests.get(BASE_URL + 'audio-features/' + track['id'], headers=headers)
        f = f.json()

        #combine with album info
        f.update({
            'track_name': track['name'],
            'album_name': album_name,
            'short_album_name': trim_name,
            'release_date': album['release_date'],
            'album_id': album['id']
        })

        data.append(f)

In [101]:
import pandas as pd

In [102]:
df = pd.DataFrame(data)

In [103]:
%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns